<a href="https://colab.research.google.com/github/nonoumasy/twitter_sent_analysis/blob/master/twitter_sent_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 3.5MB/s 


In [0]:
import tweepy
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [0]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [0]:
#vaderSentiment
analyzer = SentimentIntensityAnalyzer()

def get_related_tweets(keyword):
  twitter_users = []
  twitter_user_loc = []
  twitter_user_status_count = []
  twitter_user_follower_count = []
  tweet_time = []
  tweet_string = []
  
  tweet_retweet = []

   
  for tweet in tweepy.Cursor(api.search,q=keyword, count=1000).items(1000):
          if (not tweet.retweeted) and ('RT @' not in tweet.text):
              if tweet.lang == "en":
                  twitter_users.append(tweet.user.name)
                  twitter_user_loc.append(tweet.user.location)
                  twitter_user_status_count.append(tweet.user.statuses_count)
                  twitter_user_follower_count.append(tweet.user.followers_count)
                  tweet_retweet.append(tweet.retweet_count)
                  tweet_time.append(tweet.created_at)
                  tweet_string.append(tweet.text)
                  print([tweet.user.name,tweet.created_at,tweet.text])
  df = pd.DataFrame({'name' :twitter_users,
                     'location' : twitter_user_loc, 
                     'time' : tweet_time,
                     'retweet' : tweet_retweet,
                     'status_count' : twitter_user_status_count,
                     'follow_count' : twitter_user_follower_count,
                     'tweet' : tweet_string,
                     })
  
  #using VaderSentiment
  df['vader_sentiment'] = df['tweet'].apply(lambda x : analyzer.polarity_scores(x))

  #get the compound score from the VaderSentiment dict
  df['vader_comp'] = df['vader_sentiment'].apply(lambda x: x.get('compound'))

  return df

In [144]:
df = get_related_tweets('trams')

['Malvern170', datetime.datetime(2020, 1, 28, 20, 25, 31), 'In Poland impressed by quality and price generally. Great hotels. Tonight in Poznan watching trams pass the bar at… https://t.co/JRB5q9yNDF']
['The Good Dog Guide', datetime.datetime(2020, 1, 28, 20, 25, 3), 'Step back in time at Crich Tramway Village in Derbyshire. Ride on vintage trams through the period street. A perfec… https://t.co/5WJzqaptGq']
['Steve', datetime.datetime(2020, 1, 28, 20, 24, 33), '@OldSheffield @Owlstalk Madness to get rid of the trams..... only to bring them back 30 years later!']
['Transport for London', datetime.datetime(2020, 1, 28, 20, 8, 53), "@jarod_orla Hi Jarod,\n\nA Bus &amp; Tram Pass is valid for use on London Trams but can't be used to travel on the DLR.… https://t.co/81cmgiR89v"]
['pohutukawa', datetime.datetime(2020, 1, 28, 20, 5, 49), "@jef_poskanzer @a_son_of_a_gunn I've noticed how cars will pay attention especially to objects larger than them suc… https://t.co/VZNAjAfOwn"]
['Ian Hardie

In [145]:
df.head()

,name,location,time,retweet,status_count,follow_count,tweet,vader_sentiment,vader_comp
0,Malvern170,,2020-01-28 20:25:31,0,240,44,In Poland impressed by quality and price gener...,"{'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'comp...",0.8020
1,The Good Dog Guide,UK and Southern Ireland,2020-01-28 20:25:03,0,392765,38479,Step back in time at Crich Tramway Village in ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,Steve,Cheltenham,2020-01-28 20:24:33,0,473,102,@OldSheffield @Owlstalk Madness to get rid of ...,"{'neg': 0.166, 'neu': 0.834, 'pos': 0.0, 'comp...",-0.4926
3,Transport for London,"London, England",2020-01-28 20:08:53,0,100443,2420612,"@jarod_orla Hi Jarod,\n\nA Bus &amp; Tram Pass...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,pohutukawa,New Zealand,2020-01-28 20:05:49,0,50823,651,@jef_poskanzer @a_son_of_a_gunn I've noticed h...,"{'neg': 0.08, 'neu': 0.92, 'pos': 0.0, 'compou...",-0.1027


In [146]:
#summary
vader_pos = df[df['vader_comp'] > 0.0]
vader_neg = df[df['vader_comp'] < 0.0]

print("Number of Positive Tweets", len(vader_pos))
print("Number of Negative Tweets", len(vader_neg))
print("Number of Neutral Tweets", df.shape[0] - (len(vader_neg) + len(vader_pos)))

Number of Positive Tweets 106
Number of Negative Tweets 102
Number of Neutral Tweets 141


In [149]:
df[df['vader_comp'] < 0.0]

,name,location,time,retweet,status_count,follow_count,tweet,vader_sentiment,vader_comp
2,Steve,Cheltenham,2020-01-28 20:24:33,0,473,102,@OldSheffield @Owlstalk Madness to get rid of ...,"{'neg': 0.166, 'neu': 0.834, 'pos': 0.0, 'comp...",-0.4926
4,pohutukawa,New Zealand,2020-01-28 20:05:49,0,50823,651,@jef_poskanzer @a_son_of_a_gunn I've noticed h...,"{'neg': 0.08, 'neu': 0.92, 'pos': 0.0, 'compou...",-0.1027
5,Ian Hardie,"Sunderland, England",2020-01-28 20:03:22,0,5171,467,@frenchdave69 @bodas73 @My_Metro Niche - trams...,"{'neg': 0.292, 'neu': 0.708, 'pos': 0.0, 'comp...",-0.5106
7,♡ ⁷🤖,ᴮishess✨💃💃,2020-01-28 19:56:56,0,3504,522,@pepper_tae They be fighting trains and trams ...,"{'neg': 0.263, 'neu': 0.737, 'pos': 0.0, 'comp...",-0.3612
8,Gail,Manchester,2020-01-28 19:55:46,0,462,150,@glensheader @MCRMetrolink I’m back home after...,"{'neg': 0.135, 'neu': 0.795, 'pos': 0.07, 'com...",-0.3384
...,...,...,...,...,...,...,...,...,...
330,TalkPorty~Portobello,"Portobello, Edinburgh",2020-01-28 00:15:14,0,17658,3742,@cocteautriplets the problem is Edinburgh has ...,"{'neg': 0.131, 'neu': 0.774, 'pos': 0.096, 'co...",-0.1832
334,Yarra Trams,Melbourne,2020-01-28 00:00:06,2,82685,128368,"Reminder: Due to industrial action, no trams a...","{'neg': 0.186, 'neu': 0.814, 'pos': 0.0, 'comp...",-0.4767
336,James Dick,"Manchester, UK",2020-01-27 23:49:26,0,7073,288,@nickgoff79 I used to walk to Trafford bar tra...,"{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'comp...",-0.2960
342,Chloe 🧸,Nookling Junction,2020-01-27 23:36:07,0,9495,120,So not only did trams go on strike and dump us...,"{'neg': 0.223, 'neu': 0.777, 'pos': 0.0, 'comp...",-0.6486
